In [1]:
import io 
import re 
import string 
import tensorflow as tf
import tqdm

from tensorflow.keras import Model 
from tensorflow.keras.layers import Dot, Embedding, Flatten
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [2]:
%load_ext tensorboard

In [4]:
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

In [5]:
sentence = "The wide road shimmered in the hot sun"
tokens = list(sentence.lower().split())
print(len(tokens))

8


<h1>POSITIVE SAMPLING <h2> skip-gram

In [6]:
vocab, index = {}, 0
vocab['<pad>'] = 0
for token in tokens:
    vocab[token] = index
    index +=1
vocab_size = len(vocab)
print(vocab)

{'<pad>': 0, 'the': 5, 'wide': 1, 'road': 2, 'shimmered': 3, 'in': 4, 'hot': 6, 'sun': 7}


In [7]:
inverse_vocab = {index: token for token, index in vocab.items()}
print(inverse_vocab)

{0: '<pad>', 5: 'the', 1: 'wide', 2: 'road', 3: 'shimmered', 4: 'in', 6: 'hot', 7: 'sun'}


In [8]:
example_sequence = [vocab[word] for word in tokens]
print(example_sequence)

[5, 1, 2, 3, 4, 5, 6, 7]


In [9]:
window_size = 2
positive_skip_gram, _=tf.keras.preprocessing.sequence.skipgrams(
    example_sequence,
    vocabulary_size = vocab_size,
    window_size=window_size,
    negative_samples=0
)
print(len(positive_skip_gram))

26


In [10]:
for target, context in positive_skip_gram[:]:
    print(f"({target}, {context}): ({inverse_vocab[target]}, {inverse_vocab[context]})")

(5, 6): (the, hot)
(4, 3): (in, shimmered)
(4, 5): (in, the)
(3, 4): (shimmered, in)
(6, 5): (hot, the)
(3, 5): (shimmered, the)
(4, 6): (in, hot)
(5, 3): (the, shimmered)
(3, 2): (shimmered, road)
(5, 7): (the, sun)
(5, 2): (the, road)
(6, 4): (hot, in)
(6, 7): (hot, sun)
(7, 6): (sun, hot)
(2, 1): (road, wide)
(5, 1): (the, wide)
(2, 5): (road, the)
(1, 3): (wide, shimmered)
(2, 4): (road, in)
(1, 5): (wide, the)
(3, 1): (shimmered, wide)
(1, 2): (wide, road)
(2, 3): (road, shimmered)
(4, 2): (in, road)
(7, 5): (sun, the)
(5, 4): (the, in)


<h1> NEGATIVE SAMPLING <h2> skip-gram

In [11]:
target_word, context_word = positive_skip_gram[1]

num_ns = 4

context_class = tf.reshape(tf.constant(context_word,dtype="int64"), (1,1))
negative_smapling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
    true_classes = context_class,
    num_true=1,
    num_sampled=num_ns,
    unique=True,
    range_max=vocab_size,
    seed=SEED,
    name="negative_sampling"
)
print(negative_smapling_candidates)
print([inverse_vocab[index.numpy()] for index in negative_smapling_candidates])

tf.Tensor([2 1 4 3], shape=(4,), dtype=int64)
['road', 'wide', 'in', 'shimmered']


In [50]:
negative_smapling_candidates[:]

<tf.Tensor: shape=(4,), dtype=int64, numpy=array([18, 72,  4, 27])>

In [11]:
#Construct one training example

negative_smapling_candidates = tf.expand_dims(negative_smapling_candidates, 1)

context = tf.concat([context_class, negative_smapling_candidates], 0)

label = tf.constant([1]+[0]*num_ns, dtype="int64")

target = tf.squeeze(target_word)
context = tf.squeeze(context)
label = tf.squeeze(label)

In [12]:
print(f"target_index    : {target}")
print(f"target_word     : {inverse_vocab[target_word]}")
print(f"context_indices : {context}")
print(f"context_words   : {[inverse_vocab[c.numpy()] for c in context]}")
print(f"label           : {label}")

target_index    : 6
target_word     : hot
context_indices : [5 2 1 4 3]
context_words   : ['the', 'road', 'wide', 'in', 'shimmered']
label           : [1 0 0 0 0]


In [13]:
print("target  :", target)
print("context :", context)
print("label   :", label)

target  : tf.Tensor(6, shape=(), dtype=int32)
context : tf.Tensor([5 2 1 4 3], shape=(5,), dtype=int64)
label   : tf.Tensor([1 0 0 0 0], shape=(5,), dtype=int64)


<h1> COMPILING EVERYTHING INTO ONE FUNCTION

In [14]:
sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(size=10)
print(sampling_table)

[0.00315225 0.00315225 0.00547597 0.00741556 0.00912817 0.01068435
 0.01212381 0.01347162 0.01474487 0.0159558 ]


<h2>GENERATING TRAINING DATA

In [15]:
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
    targets, contexts, labels = [], [], []
    
    sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)
    
    for sequence in tqdm.tqdm(sequences):
         
        positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
            sequence,
            vocabulary_size=vocab_size,
            sampling_table=sampling_table,
            window_size=window_size,
            negative_samples=0)
        
        for target_word, context_word in positive_skip_grams:
            context_class = tf.expand_dims(
                tf.constant([context_word],dtype="int64"),1)
            negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
                true_classes= context_class,
                num_true=1,
                num_sampled=num_ns,
                unique=True,
                range_max=vocab_size,
                seed=SEED,
                name="negative_sampling")


            negative_sampling_candidates = tf.expand_dims(
                negative_sampling_candidates, 1)

            context = tf.concat([context_class, negative_sampling_candidates], 0)
            label = tf.constant([1]+ [0]*num_ns, dtype="int64")

            targets.append(target_word)
            contexts.append(context)
            labels.append(label)
        
    return targets, contexts, labels

In [16]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')


In [17]:
with open(path_to_file) as f: 
  lines = f.read().splitlines()
for line in lines[:20]:
  print(line)

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.


In [18]:
text_ds = tf.data.TextLineDataset(path_to_file).filter(lambda x: tf.cast(tf.strings.length(x), bool))

In [19]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    return tf.strings.regex_replace(lowercase,'[%s]' % re.escape(string.punctuation), '')

vocab_size = 4096
sequence_length = 10

vectorize_layer = TextVectorization(
    standardize = custom_standardization,
    max_tokens=vocab_size,
    output_mode = 'int',
    output_sequence_length = sequence_length
)

In [20]:
vectorize_layer.adapt(text_ds.batch(1024))

In [21]:
# Save the created vocabulary for reference.
inverse_vocab = vectorize_layer.get_vocabulary()
print(inverse_vocab[:20])

['', '[UNK]', 'the', 'and', 'to', 'i', 'of', 'you', 'my', 'a', 'that', 'in', 'is', 'not', 'for', 'with', 'me', 'it', 'be', 'your']


In [22]:
text_vector_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()

In [23]:
sequences = list(text_vector_ds.as_numpy_iterator())
print(len(sequences))

32777


In [24]:
for seq in sequences[:5]:
  print(f"{seq} => {[inverse_vocab[i] for i in seq]}")

[ 89 270   0   0   0   0   0   0   0   0] => ['first', 'citizen', '', '', '', '', '', '', '', '']
[138  36 982 144 673 125  16 106   0   0] => ['before', 'we', 'proceed', 'any', 'further', 'hear', 'me', 'speak', '', '']
[34  0  0  0  0  0  0  0  0  0] => ['all', '', '', '', '', '', '', '', '', '']
[106 106   0   0   0   0   0   0   0   0] => ['speak', 'speak', '', '', '', '', '', '', '', '']
[ 89 270   0   0   0   0   0   0   0   0] => ['first', 'citizen', '', '', '', '', '', '', '', '']


In [25]:
targets, contexts, labels = generate_training_data(
    sequences=sequences,
    window_size =2,
    num_ns=4,
    vocab_size = vocab_size,
    seed=SEED
)

print(len(targets), len(contexts), len(labels))

100%|██████████| 32777/32777 [00:19<00:00, 1667.51it/s]

64689 64689 64689


In [26]:
# Configure the dataset for performance

In [27]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets,contexts),labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

<BatchDataset shapes: (((1024,), (1024, 5, 1)), (1024, 5)), types: ((tf.int32, tf.int64), tf.int64)>


In [28]:
dataset = dataset.cache().prefetch(buffer_size = AUTOTUNE)
print(dataset)

<PrefetchDataset shapes: (((1024,), (1024, 5, 1)), (1024, 5)), types: ((tf.int32, tf.int64), tf.int64)>


In [34]:
class Word2Vec(Model):
    def __init__(self, vocab_size, embedding_dim):
        super(Word2Vec, self).__init__()
        self.target_embedding = Embedding(vocab_size,
                                         embedding_dim,
                                         input_length=1,
                                         name="w2v_embedding")
        self.context_embedding = Embedding(vocab_size,
                                          embedding_dim,
                                          input_length=num_ns+1)
        self.dots = Dot(axes=(3,2))
        self.flatten = Flatten()
        
    def call(self, pair):
        target, context = pair
        word_emb = self.target_embedding(target)
        context_emb = self.context_embedding(context)
        dots = self.dots([context_emb, word_emb])
        return self.flatten(dots)
        

In [35]:
def custom_loss(x_logit, y_true):
    return tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit,labels=y_true)

In [36]:
embedding_dim = 128
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

In [37]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs2")

In [38]:
word2vec.fit(dataset, epochs=20, callbacks=[tensorboard_callback])

Epoch 1/20
63/63 [==============================] - 2s 17ms/step - loss: 1.6089 - accuracy: 0.2153
Epoch 2/20
63/63 [==============================] - 1s 15ms/step - loss: 1.5924 - accuracy: 0.5924
Epoch 3/20
63/63 [==============================] - 1s 16ms/step - loss: 1.5537 - accuracy: 0.6430
Epoch 4/20
63/63 [==============================] - 1s 18ms/step - loss: 1.4782 - accuracy: 0.5967
Epoch 5/20
63/63 [==============================] - 1s 15ms/step - loss: 1.3812 - accuracy: 0.5879
Epoch 6/20
63/63 [==============================] - 1s 13ms/step - loss: 1.2816 - accuracy: 0.6092
Epoch 7/20
63/63 [==============================] - 1s 16ms/step - loss: 1.1879 - accuracy: 0.6413
Epoch 8/20
63/63 [==============================] - 1s 18ms/step - loss: 1.1015 - accuracy: 0.6729
Epoch 9/20
63/63 [==============================] - 1s 18ms/step - loss: 1.0218 - accuracy: 0.7054
Epoch 10/20
63/63 [==============================] - 1s 18ms/step - loss: 0.9482 - accuracy: 0.7367
Epoch 11/

In [40]:
%tensorboard --logdir logs2

In [41]:
weights = word2vec.get_layer('w2v_embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

In [43]:
out_v = io.open('vectors1.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata2.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()